In [1]:
import datetime
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV

In [1]:
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import datetime
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV
import numpy as np
from scipy.stats import randint
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV

#  {'depth': 5, 'iterations': 500, 'learning_rate': 0.01}
print(" CATBOOST ".center(80, "="))

print("Loading dataset...")

df = pd.read_csv("/Users/pryda/Documents/Projets_Jedha/2_ConversioNRate/conversion_data_train.csv")

X = df.drop(["converted"], axis=1)
y = df["converted"]

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

categoricals = ["country", "new_user", "source"]

# model = CatBoostClassifier(iterations=500, learning_rate=0.01, depth=5, loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100) # poop

# model = CatBoostClassifier(loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100) # 0.777 train, 0.7517 test

# model = CatBoostClassifier(loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100, depth=9, learning_rate=0.2, rsm=1) # f1-score on train set :  0.7910958904109588
# f1-score on test set :  0.7397585592717197

# {'depth': 4, 'iterations': 244, 'learning_rate': 0.1, 'rsm': 0.16326530612244897}

cbc = CatBoostClassifier(cat_features=categoricals, loss_function="Logloss", eval_metric="F1", verbose=100)
param_grid = {
    'learning_rate': [0.1],
    'depth': [4],
    'iterations': [244],
    'rsm': [0.16326530612244897]
}

model = GridSearchCV(cbc, param_grid = param_grid, cv=5, scoring="f1", n_jobs=-1, verbose=100)

model.fit(X_train, Y_train)

Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

# Y_train_pred_proba = model.predict_proba(X_train)
# Y_test_pred_proba = model.predict_proba(X_test)

# calculate cross_val_score

#get model cv results std scores
print(model.cv_results_)
print(model.cv_results_['mean_test_score'])

# Print the tuned parameters and score
print(model.best_params_)
print(model.best_score_)

print("f1-score on train set : ", f1_score(Y_train, Y_train_pred))
print("f1-score on test set : ", f1_score(Y_test, Y_test_pred))

# print("f1-score on train proba set : ", f1_score(Y_train, Y_train_pred_proba))
# print("f1-score on test proba set : ", f1_score(Y_test, Y_test_pred_proba))

# model.save_model("catboost_v1_randomsearch")

=================================== CATBOOST ===================================
Loading dataset...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
0:	learn: 0.6474790	total: 89.5ms	remaining: 21.7s
100:	learn: 0.7657538	total: 2.19s	remaining: 3.1s
200:	learn: 0.7653806	total: 4.4s	remaining: 942ms
243:	learn: 0.7649694	total: 5.48s	remaining: 0us
{'mean_fit_time': array([13.23340111]), 'std_fit_time': array([0.13315647]), 'mean_score_time': array([0.03143668]), 'std_score_time': array([0.00682866]), 'param_depth': masked_array(data=[4],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_iterations': masked_array(data=[244],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_learning_rate': masked_array(data=[0.1],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_rsm': masked_array(data=[0.16326530612244897],
             mask=[False],
       fill_value='?',
            dtyp

In [14]:
# Concatenate our train and test set to train your best classifier on all data with labels
# X = np.append(X_train,X_test,axis=0)
# Y = np.append(Y_train,Y_test)

model.fit(X,y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
0:	learn: 0.0000000	total: 70.2ms	remaining: 17.1s
100:	learn: 0.7574618	total: 2.82s	remaining: 3.99s
200:	learn: 0.7655804	total: 5.3s	remaining: 1.13s
243:	learn: 0.7659473	total: 6.33s	remaining: 0us


GridSearchCV(cv=10,
             estimator=<catboost.core.CatBoostClassifier object at 0x1569fd430>,
             n_jobs=-1,
             param_grid={'depth': [4], 'iterations': [244],
                         'learning_rate': [0.1], 'rsm': [0.16326530612244897]},
             scoring='f1', verbose=100)

In [2]:
df_test = pd.read_csv("/Users/pryda/Documents/Projets_Jedha/2_ConversioNRate/conversion_data_test.csv")

predictions = model.predict(df_test)
print(predictions)

pred_df = pd.DataFrame(columns=['converted'], data=predictions)
# pred_df = pd.DataFrame({ "converted": np.array(predictions)[:, -1] })
pred_df.to_csv(f"conversion_data_test_predictions_LV_V9{datetime.datetime.now().timestamp()}.csv",
               index=False)

[1 0 0 ... 0 1 0]
